# **4.7 IC Deriving New Variables**

In this notebook, new variables are derived from the existing Instacart dataset (ords_prods_merge) to generate deeper insights for the client. The process includes creating product price range labels, identifying the busiest days and periods of the day, and handling anomalies to ensure data quality. Once the data is cleaned and enriched, the final dataframe is exported for further analysis.

## Table of Contents  
- [1. Import Libraries & Data](#1-import-libraries-&-data)  
- [2. Subsetting for Testing](#2-subsetting-for-testing)
- [3. Creating Product Price Labels](#3-creating-product-price-labels)  
- [4. Handling Anomalous Price Values](#4-handling-anomalous-price-values)  
- [5. Creating Busy Day Labels](#5-creating-busy-day-labels)  
- [6. Creating Busiest Days (Plural)](#6-creating-busiest-day-lables-plural)
- [7. Creating Busiest Period of the Day](#7-creating-busiest-period-of-the-day)  
- [8. Export Final Data](#8-export-final-data)  

---

## 1. Import Libraries & Data
Here, I import the required libraries:
- `pandas` and `numpy` for data handling and numerical operations
- `os` for file handling

I then bring in the main dataframe ords_prods_merge, which contains merged Instacart orders and products data.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Importing df_ords_prods_merge
path = r'/Users/yaseminmustafa/Desktop/CareerFoundry/Exercise 4/15-05-2025_Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path,"02_Data/Prepared Data/ords_prods_merge.pkl"))

In [4]:
# Check output
ords_prods_merge.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices
0,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3
1,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4
2,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6
3,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4
4,2,202279.0,prior,3.0,5.0,9.0,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7


In [5]:
# Check shape
ords_prods_merge.shape

(32404859, 15)

---

## 2. Subsetting for Testing
To make testing faster, I create a subset of 1 million rows.

In [6]:
# Create subset
df = ords_prods_merge[:1000000]

In [7]:
# Check shape
df.shape

(1000000, 15)

---

## 3. Creating Product Price Labels
I classify products into price ranges for easier analysis:

- Low-range: ≤ $5
  
- Mid-range: $5–15

  
- High-range: > $15

In [8]:
# Define a function
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [9]:
# Apply the function
df['price_range'] = df.apply(price_label, axis=1)

/var/folders/nr/1brwh_xs1n5_2h6494zpg7jm0000gn/T/ipykernel_7284/902492192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [10]:
# Identify columns
df.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range
0,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3,Mid-range product
1,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4,Mid-range product
2,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6,Low-range product
3,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4,Mid-range product
4,2,202279.0,prior,3.0,5.0,9.0,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7,Mid-range product


In [11]:
# Check shape
df.shape

(1000000, 16)

In [12]:
# Count values in new column
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    674229
Low-range product    312859
High range            12912
Name: count, dtype: int64

In [13]:
# Check max price
df['prices'].max()

99999.0

In [14]:
# If statements with the loc () function on subset 
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High_range product'

/var/folders/nr/1brwh_xs1n5_2h6494zpg7jm0000gn/T/ipykernel_7284/3066704671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High_range product'


In [15]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [16]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [17]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     674229
Low-range product     312859
High_range product     12912
Name: count, dtype: int64

In [18]:
# Check shape 
df.shape

(1000000, 17)

In [19]:
# Identify columns
df.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range,price_range_loc
0,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3,Mid-range product,Mid-range product
1,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4,Mid-range product,Mid-range product
2,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6,Low-range product,Low-range product
3,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4,Mid-range product,Mid-range product
4,2,202279.0,prior,3.0,5.0,9.0,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7,Mid-range product,Mid-range product


In [20]:
# If statements with the loc() function on entire dataframe 
ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High_range product'

In [21]:
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge ['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [22]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [23]:
ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High_range product      417678
Name: count, dtype: int64

In [24]:
# Identify columns 
ords_prods_merge.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc
0,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3,Mid-range product
1,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4,Mid-range product
2,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6,Low-range product
3,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4,Mid-range product
4,2,202279.0,prior,3.0,5.0,9.0,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7,Mid-range product


In [25]:
# Check shape 
ords_prods_merge.shape

(32404859, 16)

In [26]:
# Check max orders
ords_prods_merge['prices'].max()

99999.0

---

## 4. Handling Anomalous Price Values
Some values in prices exceed $100, which are not realistic. I replace them with NaN.

In [27]:
# Turn anomaly values into NaNs 
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [28]:
# Check max orders
ords_prods_merge['prices'].max()

25.0

---

## 5. Creating Busy Day Labels
I label each order by day of week as:
- Busiest day (0)
- Least busy (4)
- Regularly busy (all others)

In [29]:
# Frequency of orders_day_of_week
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0.0    5779087
1.0    5303718
6.0    4190948
5.0    3952326
2.0    3947564
3.0    3600589
4.0    3554531
NaN    2076096
Name: count, dtype: int64

In [30]:
# Create a for loop
result = []

for value in ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [31]:
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly 

In [32]:
# Create a new column called "busiest day"
ords_prods_merge['busiest_day'] = result

In [33]:
# Frequency of busiest day
ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    23071241
Busiest day        5779087
Least busy         3554531
Name: count, dtype: int64

In [34]:
# Identify columns 
ords_prods_merge.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day
0,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3,Mid-range product,Regularly busy
1,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4,Mid-range product,Regularly busy
2,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6,Low-range product,Regularly busy
3,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4,Mid-range product,Regularly busy
4,2,202279.0,prior,3.0,5.0,9.0,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7,Mid-range product,Regularly busy


In [35]:
# Check shape 
ords_prods_merge.shape

(32404859, 17)

---

## 6. Creating Busiest Days (Plural)
The client requested two busiest days and two slowest days:
- Busiest days: 0 and 1
- Least busy days: 3 and 4
- Regularly busy: all others

In [36]:
# Frequency of orders_day_of_week
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0.0    5779087
1.0    5303718
6.0    4190948
5.0    3952326
2.0    3947564
3.0    3600589
4.0    3554531
NaN    2076096
Name: count, dtype: int64

*Busiest days of the week are 0 and 1.*
*Slowest days of the week are 3 and 4.*

In [37]:
# Create a for loop to assign labels based on day of week
result = []

for value in ords_prods_merge["orders_day_of_week"]:
    if value in [0, 1]:
        result.append("Busiest days")
    elif value in [3, 4]:
        result.append("Least busy days")
    else:
        result.append("Regularly busy")

In [38]:
# Create a new column called "busiest days"
ords_prods_merge["busiest_days"] = result

In [39]:
# Frequency of busiest_days
ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy     14166934
Busiest days       11082805
Least busy days     7155120
Name: count, dtype: int64

*The values in the busiest_days column match the expected counts exactly, confirming the logic has been correctly implemented. NaN values (2,076,096 rows) have likely defaulted to the "Regularly busy" category.*

In [40]:
# Identify columns
ords_prods_merge.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days
0,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3,Mid-range product,Regularly busy,Regularly busy
1,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4,Mid-range product,Regularly busy,Regularly busy
2,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6,Low-range product,Regularly busy,Regularly busy
3,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4,Mid-range product,Regularly busy,Regularly busy
4,2,202279.0,prior,3.0,5.0,9.0,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7,Mid-range product,Regularly busy,Regularly busy


In [41]:
# Check shape
ords_prods_merge.shape

(32404859, 18)

---

## 7. Creating Busiest Period of the Day
Orders are grouped into periods to help the technical team prevent app freezes:
- Most orders: 10am–3pm
- Fewest orders: 12am–5am
- Average orders: all other times

In [42]:
# Frequency of order_hour_of_day
ords_prods_merge['order_hour_of_day'].value_counts(dropna = False)

order_hour_of_day
10.0    2593725
11.0    2564597
14.0    2517238
15.0    2487586
13.0    2487500
12.0    2445841
16.0    2364969
9.0     2311334
NaN     2076096
17.0    1943858
8.0     1622394
18.0    1520954
19.0    1169224
20.0     910005
7.0      844665
21.0     746254
22.0     592432
23.0     375889
6.0      274801
0.0      203460
1.0      108110
5.0       82706
2.0       63961
4.0       49400
3.0       47860
Name: count, dtype: int64

In [43]:
# Create a for loop to assign labels based on hours of week
result = []

for value in ords_prods_merge["order_hour_of_day"]:
    if value in [10, 11, 14, 15, 13, 12]:
        result.append("Most orders")
    elif value in [1, 2, 3, 4, 5, 0]:
        result.append("Fewest orders")
    else:
        result.append("Average orders")

In [44]:
# Create a new column called "busiest_period_of_day"
ords_prods_merge["busiest_period_of_day"] = result

In [45]:
# Frequency of busiest_period_of_day
ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Average orders    16752875
Most orders       15096487
Fewest orders       555497
Name: count, dtype: int64

In [46]:
# Identify columns 
ords_prods_merge.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4,Mid-range product,Regularly busy,Regularly busy,Average orders
2,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6,Low-range product,Regularly busy,Regularly busy,Average orders
3,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4,Mid-range product,Regularly busy,Regularly busy,Average orders
4,2,202279.0,prior,3.0,5.0,9.0,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7,Mid-range product,Regularly busy,Regularly busy,Average orders


In [47]:
# Check shape
ords_prods_merge.shape

(32404859, 19)

---

## 8. Export Final Data
The enriched dataframe is exported as a pickle file for later use.

In [48]:
# Export data to pkl
ords_prods_merge.to_pickle(os.path.join(path, '02_Data','Prepared Data', 'ords_prods_merge_new_variables.pkl'))